In [1]:
include("spiking.jl")

train_to_phase (generic function with 1 method)

In [2]:
using Plots

In [3]:
phase = rand(Float32, 128, 100) .* 2 .- 1

128×100 Matrix{Float32}:
  0.991597    0.426675   -0.362095   …  -0.957286    -0.22808    0.33551
 -0.0295937  -0.729119   -0.334633      -0.4139      -0.739517  -0.210165
  0.26        0.336424   -0.55741        0.769932    -0.935535  -0.214242
 -0.227017   -0.616549    0.400911      -0.650808    -0.21947   -0.316521
 -0.0774051   0.534747    0.992922      -0.00634348  -0.745821  -0.42693
 -0.683507   -0.408102   -0.898305   …   0.449537    -0.572226   0.924955
 -0.261005   -0.510529   -0.581282       0.526836     0.248605   0.662258
  0.668194   -0.873789   -0.862615      -0.620982     0.499545   0.031122
  0.0461512  -0.64515     0.853326      -0.14        -0.194399   0.239467
 -0.128273    0.583601   -0.205623      -0.0859361    0.510574  -0.453733
  0.873012   -0.411573   -0.117069   …   0.925442     0.839048  -0.95957
 -0.600245    0.27331     0.798557      -0.228938     0.513269   0.252239
 -0.983132    0.574857   -0.79492        0.537614    -0.480118   0.351033
  ⋮             

In [10]:
spk_args = default_spk_args()

SpikingArgs(6.283185307179586, -0.2, 1.0, 0.03, 0.05)

In [20]:
function phase_to_train(phases::AbstractMatrix, spk_args::SpikingArgs, repeats::Int = 1, offset::Real = 0.0)
    t_phase0 = spk_args.t_period / 2.0
    shape = phases |> size
    indices = collect(CartesianIndices(shape)) |> vec
    times = phases .* t_phase0 .+ t_phase0 |> vec

    if repeats > 1
        n_t = times |> length
        offsets = repeat(0:repeats-1, inner=n_t)
        times = repeat(times, repeats) .+ offsets
        indices = repeat(indices, repeats)
    end

    train = SpikeTrain(indices, times, shape, offset)
    return train
end

phase_to_train (generic function with 3 methods)

In [28]:
st = phase_to_train(phase, spk_args, 2, 0.0)

SpikeTrain(CartesianIndex{2}[CartesianIndex(1, 1), CartesianIndex(2, 1), CartesianIndex(3, 1), CartesianIndex(4, 1), CartesianIndex(5, 1), CartesianIndex(6, 1), CartesianIndex(7, 1), CartesianIndex(8, 1), CartesianIndex(9, 1), CartesianIndex(10, 1)  …  CartesianIndex(119, 100), CartesianIndex(120, 100), CartesianIndex(121, 100), CartesianIndex(122, 100), CartesianIndex(123, 100), CartesianIndex(124, 100), CartesianIndex(125, 100), CartesianIndex(126, 100), CartesianIndex(127, 100), CartesianIndex(128, 100)], [0.995798647403717, 0.4852031469345093, 0.6299999356269836, 0.38649141788482666, 0.4612974524497986, 0.15824639797210693, 0.3694975972175598, 0.8340970277786255, 0.5230755805969238, 0.435863733291626  …  1.4890093207359314, 1.3397400975227356, 1.963085114955902, 1.227163314819336, 1.702117681503296, 1.7649328708648682, 1.497108280658722, 1.2378676533699036, 1.7786964178085327, 1.8653514981269836], (128, 100), 0.0)

In [29]:
c = train_to_phase(st, spk_args)

2×128×100 Array{Float64, 3}:
[:, :, 1] =
 0.991597  -0.0295937  0.26  -0.227017  …  -0.369217  0.940975  0.308368
 0.991597  -0.0295937  0.26  -0.227017     -0.369217  0.940975  0.308368

[:, :, 2] =
 0.426675  -0.729119  0.336424  -0.616549  …  -0.133853  0.172427  -0.296782
 0.426675  -0.729119  0.336424  -0.616549     -0.133853  0.172427  -0.296782

[:, :, 3] =
 -0.362095  -0.334633  -0.55741  0.400911  …  -0.48012  0.522655  0.0995046
 -0.362095  -0.334633  -0.55741  0.400911     -0.48012  0.522655  0.0995046

;;; … 

[:, :, 98] =
 -0.957286  -0.4139  0.769932  -0.650808  …  -0.595258  0.572075  -0.902339
 -0.957286  -0.4139  0.769932  -0.650808     -0.595258  0.572075  -0.902339

[:, :, 99] =
 -0.22808  -0.739517  -0.935535  -0.21947  …  -0.562278  -0.399979  0.773259
 -0.22808  -0.739517  -0.935535  -0.21947     -0.562278  -0.399979  0.773259

[:, :, 100] =
 0.33551  -0.210165  -0.214242  -0.316521  …  -0.524265  0.557393  0.730703
 0.33551  -0.210165  -0.214242  -0.316521     -0

In [31]:
c[1,:,:] .== c[2,:,:]

128×100 BitMatrix:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1 

In [32]:
phase .== c[1,:,:]

128×100 BitMatrix:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1 

In [ ]:
function time_to_phase(times::AbstractVecOrMat, period::Real, offset::Real)
    times = (times .- offset) .% period
    times = (times .- 0.5) .* 2.0
    return times
end

function train_to_phase(train::SpikeTrain, spk_args::SpikingArgs)
    if length(train.times) == 0
        return missing
    end

    phases_vec = time_to_phase(train.times, spk_args.t_period, train.offset)
    n_cycles = maximum(train.times) ÷ spk_args.t_period + 1
    cycle = floor.(Int, train.times .÷ spk_args.t_period .+ 1)
    phases = [NaN .* zeros(train.shape...) for i in 1:n_cycles]

    for i in eachindex(phases_vec)
        phases[cycle[i]][train.indices[i]] = phases_vec[i]
    end

    #stack the arrays
    phases = mapreduce(x->reshape(x, 1, st.shape...), vcat, phases)
    phases = permutedims(phases, (2, 3, 1))
    return phases

end

train_to_phase (generic function with 2 methods)